In [8]:
import pandas as pd
import numpy as np
from glob import glob

df = pd.read_table('/home/groups/dpwall/briannac/y_chromosome_mismappings/intermediate_files/coverages/chrom_lengths.tsv', header=None)

In [9]:
df['chrom'] = [i.replace('SN:', '') for i in df[1]]
df['length'] = [int(i.replace('LN:', '')) for i in df[2]]
df.drop([0,1,2], inplace=True, axis=1)

In [10]:
df['start'] = [0] + list(np.cumsum(df['length'])[:-1].values)
df['finish'] = np.cumsum(df['length'])-1
df.index = df.chrom
df = df[['start', 'finish', 'length']]

In [11]:
df.index = ['chrDecoy' if i==25 else idx for i,idx in enumerate(df.index)]
df = df.iloc[:26]
df

,start,finish,length
chr1,0,248956421,248956422
chr2,248956422,491149950,242193529
chr3,491149951,689445509,198295559
chr4,689445510,879660064,190214555
chr5,879660065,1061198323,181538259
chr6,1061198324,1232004302,170805979
chr7,1232004303,1391350275,159345973
chr8,1391350276,1536488911,145138636
chr9,1536488912,1674883628,138394717
chr10,1674883629,1808681050,133797422


In [5]:
df.to_csv('/home/groups/dpwall/briannac/y_chromosome_mismappings/intermediate_files/coverages/chrom_start_stops.tsv',
         sep='\t')

In [6]:
mylist = []
for row in df.iterrows():
    chrom=row[0]
    for i, start in enumerate(np.arange(row[1].start, row[1].finish,1000000)):
        if start >= row[1].finish: break
        idx = chrom + '_' + str(i)
        mylist = mylist + [(idx, start, min(row[1].finish, start+1000000))]

In [9]:
new_df = pd.DataFrame(mylist)
new_df.index = new_df[0]
new_df.drop(0, axis=1, inplace=True)
new_df.to_csv('/home/groups/dpwall/briannac/y_chromosome_mismappings/intermediate_files/coverages/chrom_start_stops_intervals.tsv',
         sep='\t')

In [26]:
import pandas as pd
from glob import glob
files = glob('/home/groups/dpwall/briannac/y_chromosome_mismappings/intermediate_files/covWAS/*.done')
files = [f.split('/')[-1].replace('.done', '') for f in files]

starts_all = pd.read_table('/home/groups/dpwall/briannac/y_chromosome_mismappings/intermediate_files/coverages/chrom_start_stops_intervals.tsv', index_col=0)
starts_all.loc[[i for i in starts_all.index if i not in files]].to_csv(
    '/home/groups/dpwall/briannac/y_chromosome_mismappings/intermediate_files/coverages/chrom_start_stops_intervals_unfinished.tsv',
    sep='\t')